In [21]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from matplotlib import cm
from matplotlib.colors import to_hex

# Read the TSV file
unbounded = pd.read_csv("bench_unbounded.tsv", sep="\t")

# Convert numeric columns for unbounded dataset
unbounded['query_len'] = pd.to_numeric(unbounded['query_len'])
unbounded['ref_len'] = pd.to_numeric(unbounded['ref_len'])
unbounded['run_time'] = pd.to_numeric(unbounded['run_time'])

# Create pivot table for speedup calculation
pivot_unbounded = unbounded.pivot_table(index=["query_len", "ref_len"], columns="tool", values="run_time").reset_index()
pivot_unbounded['speedup_unbounded'] = pivot_unbounded['edlib'] / pivot_unbounded['sassy']

merged = pivot_unbounded[['query_len', 'ref_len', 'speedup_unbounded']]

query_lens = sorted(merged['query_len'].unique())
cmap = cm.get_cmap('turbo', len(query_lens))
color_map = {qlen: to_hex(cmap(i)) for i, qlen in enumerate(query_lens)}

fig = go.Figure()

for qlen in query_lens:
    sub = merged[merged['query_len'] == qlen]
    x = sub['ref_len'].values
    color = color_map[qlen]

    # Unbounded speedup
    fig.add_trace(go.Scatter(
        x=x,
        y=sub['speedup_unbounded'],
        mode='markers+lines',  # Added lines between points
        marker=dict(
            color=color,
            symbol='square',
            size=10,
            line=dict(width=1, color='black')
        ),
        line=dict(
            color=color,
            dash='dash',  # Make lines dashed
            width=2
        ),
        name=f"Query {qlen}",
        legendgroup=str(qlen),
        showlegend=True
    ))

fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers+lines',  # Updated to match the main traces
    marker=dict(symbol='square', color='gray', size=10, line=dict(color='black')),
    line=dict(color='gray', dash='dash', width=2),  # Added dashed line to legend
    name='Unbounded Speedup'
))

# Layout
fig.update_layout(
    title="Sassy Speedup over Edlib: Unbounded",
    xaxis_title="Reference Length (log scale)",
    yaxis_title="Speedup (Edlib time / Sassy time)",
    xaxis=dict(type='log', tickvals=sorted(merged['ref_len'].unique())),
    yaxis=dict(type='log'),
    template="plotly_white",
    font=dict(size=16),
    height=700,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.3,
        xanchor="center",
        x=0.5,
        title=None
    )
)

# Add 1X baseline
fig.add_hline(y=1, line_dash="dot", line_color="gray")

fig.show()


/tmp/ipykernel_417465/3189833657.py:22: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.

